# Constructing Pauli Twirling Sets from Arbitrary Error Channels


"Twirling is a technique widely used for converting arbitrary noise channels into Pauli channels in error threshold estimations of quantum error correction codes. It is vitally useful both in real experiments and in classical quantum simulations. Minimising the size of the twirling gate set increases the efficiency of simulations and in experiments it might reduce both the number of runs required and the circuit depth (and hence the error burden). Conventional twirling uses the full set of Pauli gates as the set of twirling gates. This article provides a theoretical background for Pauli twirling and a way to construct a twirling gate set with a number of members comparable to the size of the Pauli basis of the given error channel, which is usually much smaller than the full set of Pauli gates. We also show that twirling is equivalent to stabiliser measurements with discarded measurement results, which enables us to further reduce the size of the twirling gate set."